In [1]:
import pandas as pd
import sqlite3

In [2]:
query = '''
WITH hospitals_tn AS(
    SELECT 
        npi,
        entity_type_code AS entity,
        `provider_organization_name_(legal_business_name)` AS organization,
        SUBSTR(provider_business_practice_location_address_postal_code, 1, 5) AS zipcode,
        taxonomy_code
    FROM npi_data
    WHERE 
        entity_type_code = 2
)
SELECT
    from_tax.grouping AS from_grouping,
    from_tax.classification AS from_classification,
    from_tax.specialization AS from_specialization,
    to_npi,
    htn.organization AS to_organization,
    to_tax.grouping AS to_grouping,
    to_tax.classification AS to_classification,
    to_tax.specialization AS to_specialization,
    SUM(patient_count) AS total_patient
FROM doc_graph
INNER JOIN hospitals_tn AS htn
ON to_npi = htn.npi
INNER JOIN npi_data AS fro
ON from_npi = fro.npi
LEFT JOIN taxonomy AS from_tax
ON fro.taxonomy_code = from_tax.code
LEFT JOIN taxonomy AS to_tax
ON htn.taxonomy_code = to_tax.code
WHERE htn.zipcode IN (
    SELECT zip
    FROM zipcode
    WHERE cbsa = 34980
)
GROUP BY 
    from_tax.grouping,
    from_tax.specialization,
    from_tax.classification,
    to_npi,
    htn.organization,
    to_tax.grouping,
    to_tax.classification,
    to_tax.specialization
ORDER BY total_patient DESC
'''

In [3]:
with sqlite3.connect('../data/referrals.sqlite') as db: 
    referrals = pd.read_sql(query, db)

In [18]:
referrals.to_csv('../data/referrals.csv', index = False)

In [5]:
query = '''
WITH hospitals_tn AS(
    SELECT 
        npi,
        entity_type_code AS entity,
        `provider_organization_name_(legal_business_name)` AS organization,
        SUBSTR(provider_business_practice_location_address_postal_code, 1, 5) AS zipcode,
        taxonomy_code
    FROM npi_data
    WHERE 
        entity_type_code = 2
)
SELECT
    from_npi,
    fro.`provider_organization_name_(legal_business_name)` AS from_organization,
    from_tax.grouping AS from_grouping,
    from_tax.classification AS from_classification,
    from_tax.specialization AS from_specialization,
    to_npi,
    htn.organization AS to_organization,
    to_tax.grouping AS to_grouping,
    to_tax.classification AS to_classification,
    to_tax.specialization AS to_specialization,
    patient_count
FROM doc_graph
INNER JOIN hospitals_tn AS htn
ON to_npi = htn.npi
INNER JOIN npi_data AS fro
ON from_npi = fro.npi
LEFT JOIN taxonomy AS from_tax
ON fro.taxonomy_code = from_tax.code
LEFT JOIN taxonomy AS to_tax
ON htn.taxonomy_code = to_tax.code
WHERE htn.zipcode IN (
    SELECT zip
    FROM zipcode
    WHERE cbsa = 34980
) AND
    lower(to_tax.grouping) LIKE '%hospital%'
'''

In [6]:
with sqlite3.connect('../data/referrals.sqlite') as db: 
    hos_referrals = pd.read_sql(query, db)

In [11]:
hos_referrals.to_csv('../data/hos_referrals.csv', index = False)

In [5]:
query = '''
WITH hospitals_tn AS(
    SELECT 
        npi,
        entity_type_code AS entity,
        `provider_organization_name_(legal_business_name)` AS organization,
        SUBSTR(provider_business_practice_location_address_postal_code, 1, 5) AS zipcode,
        taxonomy_code
    FROM npi_data
    WHERE 
        entity_type_code = 2
)
SELECT
    from_npi,
    fro.`provider_organization_name_(legal_business_name)` AS from_organization,
    to_npi,
    htn.organization AS to_organization,
    patient_count
FROM doc_graph
INNER JOIN hospitals_tn AS htn
ON to_npi = htn.npi
INNER JOIN npi_data AS fro
ON from_npi = fro.npi
LEFT JOIN taxonomy AS from_tax
ON fro.taxonomy_code = from_tax.code
LEFT JOIN taxonomy AS to_tax
ON htn.taxonomy_code = to_tax.code
WHERE htn.zipcode IN (
    SELECT zip
    FROM zipcode
    WHERE cbsa = 34980
)
'''

In [ ]:
with sqlite3.connect('../data/referrals.sqlite') as db: 
    communities = pd.read_sql(query, db)

In [6]:
communities.to_csv('../data/communities.csv', index = False)

In [7]:
pd.read_csv('../data/communities.csv')

,from_npi,from_organization,to_npi,to_organization,patient_count
0,1528053311,"AMERICAN ENDOSCOPY CENTER, P.C.",1518952308,"NASHVILLE DIGESTIVE DISEASE CENTER, P.C.",43
1,1003863580,"ASSOCIATED PATHOLOGISTS, LLC",1518952308,"NASHVILLE DIGESTIVE DISEASE CENTER, P.C.",54
2,1003801937,NaN,1518952308,"NASHVILLE DIGESTIVE DISEASE CENTER, P.C.",96
3,1093753303,NaN,1518952308,"NASHVILLE DIGESTIVE DISEASE CENTER, P.C.",44
4,1093741464,"ADVANCED DIAGNOSTIC IMAGING, PC",1518952308,"NASHVILLE DIGESTIVE DISEASE CENTER, P.C.",38
...,...,...,...,...,...
124081,1922057181,MID-TENNESSEE BONE & JOINT,1104850130,HEIKKI E KOSTAMAA MD PC,43
124082,1700878360,"NEPHROLOGY ASSOCIATES, PC",1104850130,HEIKKI E KOSTAMAA MD PC,76
124083,1861479545,MAURY REGIONAL HOSPITAL,1104850130,HEIKKI E KOSTAMAA MD PC,195
124084,1205886264,"MAURY REGIONAL MEDICAL GROUP, INC",1104850130,HEIKKI E KOSTAMAA MD PC,146


In [12]:
pd.read_csv('../data/hos_referrals.csv')

,from_npi,from_organization,from_grouping,from_classification,from_specialization,to_npi,to_organization,to_grouping,to_classification,to_specialization,patient_count
0,1518143122,NaN,Allopathic & Osteopathic Physicians,Family Medicine,NaN,1265445506,WILLIAMSON COUNTY HOSPITAL DISTRICT,Hospitals,General Acute Care Hospital,NaN,190
1,1518122613,NaN,Allopathic & Osteopathic Physicians,Internal Medicine,Clinical Cardiac Electrophysiology,1265445506,WILLIAMSON COUNTY HOSPITAL DISTRICT,Hospitals,General Acute Care Hospital,NaN,56
2,1518078039,NaN,Podiatric Medicine & Surgery Service Providers,Podiatrist,NaN,1265445506,WILLIAMSON COUNTY HOSPITAL DISTRICT,Hospitals,General Acute Care Hospital,NaN,90
3,1518285659,NaN,Allopathic & Osteopathic Physicians,Emergency Medicine,NaN,1265445506,WILLIAMSON COUNTY HOSPITAL DISTRICT,Hospitals,General Acute Care Hospital,NaN,252
4,1518945195,NaN,Allopathic & Osteopathic Physicians,Psychiatry & Neurology,Psychiatry,1265445506,WILLIAMSON COUNTY HOSPITAL DISTRICT,Hospitals,General Acute Care Hospital,NaN,71
...,...,...,...,...,...,...,...,...,...,...,...
14524,1093741464,"ADVANCED DIAGNOSTIC IMAGING, PC",Allopathic & Osteopathic Physicians,Internal Medicine,NaN,1679924005,"CURAHEALTH NASHVILLE, LLC",Hospitals,Long Term Care Hospital,NaN,33
14525,1922331842,"PHYSICIAN SERVICES OF MIDDLE TENNESSEE, LLC",Allopathic & Osteopathic Physicians,Internal Medicine,NaN,1679924005,"CURAHEALTH NASHVILLE, LLC",Hospitals,Long Term Care Hospital,NaN,67
14526,1952361065,"NASHVILLE LUNG CENTER, INC",Allopathic & Osteopathic Physicians,Internal Medicine,Pulmonary Disease,1679924005,"CURAHEALTH NASHVILLE, LLC",Hospitals,Long Term Care Hospital,NaN,45
14527,1992039184,XUHAN PC LLP,Allopathic & Osteopathic Physicians,Internal Medicine,NaN,1679924005,"CURAHEALTH NASHVILLE, LLC",Hospitals,Long Term Care Hospital,NaN,40


In [19]:
pd.read_csv('../data/referrals.csv')

,from_grouping,from_classification,from_specialization,to_npi,to_organization,to_grouping,to_classification,to_specialization,total_patient
0,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,1124075635,ASSOCIATED PATHOLOGISTS LLC,Laboratories,Clinical Medical Laboratory,NaN,389861
1,Laboratories,Clinical Medical Laboratory,NaN,1003863580,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,330083
2,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,1003863580,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,242187
3,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,1235186800,"PATHGROUP LABS, LLC",Laboratories,Clinical Medical Laboratory,NaN,239242
4,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,1861478489,RADIOLOGY ALLIANCE PC,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,180474
...,...,...,...,...,...,...,...,...,...
24046,Suppliers,Durable Medical Equipment & Medical Supplies,NaN,1366554180,"DIALYSIS ASSOCIATES, LLC",Ambulatory Health Care Facilities,Clinic/Center,End-Stage Renal Disease (ESRD) Treatment,11
24047,Suppliers,Pharmacy,Community/Retail Pharmacy,1457303083,TENNESSEE ONCOLOGY PLLC,Allopathic & Osteopathic Physicians,Internal Medicine,Hematology & Oncology,11
24048,Suppliers,Durable Medical Equipment & Medical Supplies,Parenteral & Enteral Nutrition,1699720029,NATIONAL HEALTH CORPORATION,Nursing & Custodial Care Facilities,Skilled Nursing Facility,NaN,11
24049,Suppliers,Durable Medical Equipment & Medical Supplies,Parenteral & Enteral Nutrition,1730134610,NHC HEALTHCARE-MCMINNVILLE LLC,Nursing & Custodial Care Facilities,Skilled Nursing Facility,NaN,11


In [3]:
pd.read_csv('../data/neo_communities.csv')

,npi,communityId
0,1003001066,2210
1,1003001363,1262
2,1003009846,1262
3,1003009861,29892
4,1003010117,29925
...,...,...
30123,1992985949,195
30124,1992986749,2210
30125,1992987085,268
30126,1992988257,1262
